In [ ]:
import pandas as pd

In [ ]:
def fix_smiles(smi_in):
    return smi_in.strip('[]').strip("'")

In [ ]:
df_list = []
for split in ["random","scaffold"]:
    for task in ["st","mt"]:
        for i in range(0,10):
            dirname = f"data/BSEP{i:03d}"
            if task == "st":
                df_test = pd.read_csv(f"{dirname}/{split}_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_ST/test_preds.csv")
            else:
                df_test = pd.read_csv(f"{dirname}/{split}_mt_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_MT/test_preds.csv")
            df_pred['SMILES'] = df_pred.smiles.apply(fix_smiles)
            df_pred.drop('smiles',axis=1,inplace=True)
            df_pred.rename(columns={"BSEP": "BSEP_prob"},inplace=True)
            df_test.BSEP = df_test.BSEP.astype(int)
            df_combo = pd.concat([df_test[["SMILES","Name","BSEP"]],df_pred[["BSEP_prob"]]],axis=1)
            df_combo['BSEP_pred'] = [1 if p > 0.5 else 0 for p in df_combo.BSEP_prob]
            df_combo['cv_cycle'] = i
            df_combo['split'] = split
            df_combo['method'] = f"chemprop_{task}"
            df_combo = df_combo[["cv_cycle","split","method","SMILES","Name","BSEP","BSEP_prob","BSEP_pred"]]
            df_list.append(df_combo)

In [ ]:
df_out = pd.concat(df_list)
df_out.to_csv("BSEP_classification_preds.csv",index=False)

In [ ]:
df_out